In [21]:
## imports for custom overlay
from pynq import Overlay
## Import for DMA alloc
from pynq import allocate
## import for video driver API support
from pynq.lib.video import *
## import for timing
import time
## import for rectangles and any opencv we do in software
import cv2

## CUSTOM OVERLAY 
hudson_overlay = Overlay('movingrect.bit')
# DMA vars
vdma = hudson_overlay.video.axi_vdma
dma = hudson_overlay.redDMA.axi_dma_0
# config axi lite access
axilite = hudson_overlay.redDMA.threshold_accel_0

In [22]:
# send input buffer and read the output

def run_kernel():
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()

In [23]:
# Set HDMI in and out vars
hdmi_in = hudson_overlay.video.hdmi_in
hdmi_out = hudson_overlay.video.hdmi_out
# initialize IO
hdmi_in.configure(PIXEL_RGB)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGB)



In [24]:
hdmi_in.start()
hdmi_out.start()

In [25]:
# Read in a frame and allocate space for it in
# what will become axi stream buffer
shaped_in = hdmi_in.readframe()
in_buffer = allocate(shape=(720,1280,3), dtype=np.uint8)
out_buffer = allocate(shape=(720,1280,3), dtype=np.uint8)

# show on the screen what an in buffer frame looks like
in_buffer[:] = shaped_in
hdmi_out.writeframe(in_buffer)

In [26]:
# set resolution rows and cols via axi lite
axilite.write(0x10, 30) # thresh
axilite.write(0x18, 255) # maxval
axilite.write(0x20, 720) # rows
axilite.write(0x28, 1280) # cols

In [17]:
# point HDMI out to the in buffer
hdmi_out.writeframe(in_buffer)

size = 200

x = 0
y = 0

numframes = 500
start = time.time()

# time <numframes> frames of processing
for _ in range(numframes):
    # read a frame into buffer to hold it
    in_buffer = hdmi_in.readframe()
    # send in buffer to hardware for processing
    run_kernel()
    # read the axi lite ports for rectangle coordinates and put them on the in_buffer
    cv2.rectangle(in_buffer, (axilite.read(0x30),axilite.read(0x38)), (x + size, y + size), (0, 0, 255), 2)
    # write buffer to screen
    # NOTE: change this line to write out buffer to see what the processed image looks like
    #       or in buffer to see the original image with recatagles drawn
    hdmi_out.writeframe(out_buffer)
    
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

    

KeyboardInterrupt: 

20 frames per second when thresholding in hardware

In [27]:
in_buffer = hdmi_in.readframe()

In [28]:
run_kernel()

KeyboardInterrupt: 

In [29]:

hdmi_in.close()
hdmi_out.close()